In [ ]:
import os
import re
import json
import boto3
import requests
from dotenv import load_dotenv

load_dotenv()
AWS_ACCESS_KEY_ID = os.environ.get('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.environ.get('AWS_SECRET_ACCESS_KEY')
REGION_NAME = os.environ.get('REGION_NAME')

dynamodb = boto3.resource('dynamodb', aws_access_key_id=AWS_ACCESS_KEY_ID, aws_secret_access_key=AWS_SECRET_ACCESS_KEY, region_name=REGION_NAME)

access_token = 'BQDPD4NGFRIUSOyOgYJDQ2X_LTrYOKD27a6oM1ShJxvZq-H6rR5pazZYejWGzCfsKZvuUbWqBU8slfdKWPDnJLkKuC9ftGD6629lS2BpRbedtnA2dd8T1SiPM22coaN0sR8bGCOHdKWUrVQUW2Fh5ToGI1EnwuAsISHu5e8kR5oXihULMwVVNiL-qd6PROfynz_0mRRjwNa3Of0TnK7QIA'



In [74]:
# Helper functions:

def get_playlist_uri(playlist_url: str):
    return re.search(r'/([^/]+)\?', playlist_url).group(1)

def get_track_ids_and_store_audio_features(access_token: str, playlist_uri: str):
    playlist_info = {
        'track_ids': [],
        'playlist_uri': playlist_uri,
    }

    tracks_table = dynamodb.Table('SPOTIFY_TRACKS')

    headers = {
        'Authorization': f"Bearer {access_token}"
    }

    print(f"fetching playlist {playlist_uri}")
    raw_json_to_check = {'next': f"https://api.spotify.com/v1/playlists/{playlist_uri}/tracks?market=US&limit=50"}

    while raw_json_to_check['next'] != None:
        response = requests.get(url = raw_json_to_check['next'], headers = headers)

        if response.status_code == 200:
            playlist_raw_json = response.json()
            
            track_ids = []
            for item in playlist_raw_json['items']:
                if item['track'] != None:
                    track_id = item['track']['id']
                    playlist_info['track_ids'].append(track_id)

                    db_response = tracks_table.get_item(Key={
                        'id': track_id,
                    })

                    # add track if havent been added
                    if ('Item' not in db_response):
                        track_ids.append(track_id)


            query = {
                'ids': ','.join(track_ids)
            }

            if len(track_ids) != 0:
                response = requests.get(url = 'https://api.spotify.com/v1/audio-features', headers = headers, params = query)

                if response.status_code == 200:
                    raw_json = response.json()
                    
                    i = 0
                    while i < len(raw_json['audio_features']):
                        audio_features = raw_json['audio_features'][i]

                        if audio_features != "None" and audio_features != None:
                            for key, value in audio_features.items():
                                if isinstance(value, float):
                                    raw_json['audio_features'][i][key] = str(value)
                                    
                        else:
                            raw_json['audio_features'].pop(i)
                            i -= 1

                        i += 1
                    
                    for track in raw_json['audio_features']:
                        tracks_table.put_item(Item=track)

            raw_json_to_check = playlist_raw_json
    
    return playlist_info

In [75]:
playlists = [
    ['https://open.spotify.com/playlist/7BIFvKEvSWfr0c1Ur2PxQ5?si=34101201091d4f3f', 'https://open.spotify.com/playlist/6w2nktImGPS83JiAkG2VzB?si=c6539d4ff9f942bf', 'https://open.spotify.com/playlist/5NZj30WhMY4gnMIGPD8gir?si=24cc7fa618754012'],
    ['https://open.spotify.com/playlist/7LDJsFK7uk53RNxax3z2YC?si=efa7f1f9e09a4268', 'https://open.spotify.com/playlist/0eN9zu6ZezHsNodVtHIlAU?si=34d8625b252e4c45', 'https://open.spotify.com/playlist/6unGtxWr0zGP2GJQijrzTW?si=253b268341974cce'],
    ['https://open.spotify.com/playlist/3kTgfkfU6GomvrEyNgC7LV?si=8490e9c85b8242ff', 'https://open.spotify.com/playlist/6SP3BZKRwpp5PX0rI1Rmem?si=d121334278574748', 'https://open.spotify.com/playlist/1AmBxyen2AaSKf6xgl4FAa?si=0e29e78eb1da4fff'],
    ['https://open.spotify.com/playlist/6zut6b4UaLmsxVPvTHAyUC?si=f1347e12a7e34a0d', 'https://open.spotify.com/playlist/3n5J91l343pfMflirtBZhD?si=0765cda31d53453f', 'https://open.spotify.com/playlist/5Wlx45esaqtfimuHPGYaF4?si=a86f7f3b46d346c7'],
    ['https://open.spotify.com/playlist/3GzH2VIpDXQaouhWJeBvYn?si=d2f3001e9a754ca4', 'https://open.spotify.com/playlist/4OUEfwVUeRHzAKW5rGjH3M?si=0c22d35408734834', 'https://open.spotify.com/playlist/6XmxwdTuFrNQJjhUOEPki3?si=476b15ffd94a4995'],
    ['https://open.spotify.com/playlist/3thnEzH8MTVm5t4kyyNVGV?si=755d65300bc64b75', 'https://open.spotify.com/playlist/2k2n89VO9E6uYsxPvatLsk?si=9bfe933f2345489a', 'https://open.spotify.com/playlist/4ytPJrzShCo3iftbTgHi4p?si=8bbde85ad4c44980'],
    ['https://open.spotify.com/playlist/0c9nCMbp3TcR8aLbsyn30N?si=13ba22eb0dab44ba', 'https://open.spotify.com/playlist/5kjhVUgYNyu7zZ1rVEgnAm?si=736fffa3ffd74542', 'https://open.spotify.com/playlist/4MJf4EASw6fzzfJlUFv5l0?si=69b9e7f865114349'],
    ['https://open.spotify.com/playlist/3V2DajyxOv0Nnfk0AwjcxV?si=06c7f948c9444d3d', 'https://open.spotify.com/playlist/2NesB7u53TUuMUTUrkntLL?si=95cd05d369c04c6a', 'https://open.spotify.com/playlist/0ybME665syNkRpRUIRcKqj?si=2a728d1376d448bb'],
    ['https://open.spotify.com/playlist/34FE23iG477W2CrtcCw6NM?si=bc0fd78a986a4e03', 'https://open.spotify.com/playlist/5W4HtI15pWZLiALqm0XWhb?si=e6a3ffb7fda44c61', 'https://open.spotify.com/playlist/4OrOYDkvh9bZ0apjdbXnDd?si=c9127ffd33e843d5'],
    ['https://open.spotify.com/playlist/6q8Uqvbgf01UFYK3NjgKOa?si=2f5c2e1a201244d4', 'https://open.spotify.com/playlist/2MQ1o8Fg21lHJjRTrRzyfl?si=8dc19716b3ed4c24', 'https://open.spotify.com/playlist/1UBn0YJoWBgy93m94mxRpC?si=09dd06b02fd94314'],
    ['https://open.spotify.com/playlist/44V6F2OpcEgCySkClW5G0n?si=be620ba860974eb9', 'https://open.spotify.com/playlist/1b6TtvoM7h8LqF9d9mKYHg?si=5167bfce7ee54cae', 'https://open.spotify.com/playlist/34TkiR7rBPL38HiUhIAja2?si=698396ac02b04a4d'],
    ['https://open.spotify.com/playlist/3PoCb5MYz0AoDNf4iQvTuW?si=10fdb4af683a4d3f', 'https://open.spotify.com/playlist/27ESv3ew8e6x4t0CFADH8W?si=7e11628658164d1e', 'https://open.spotify.com/playlist/3se77cC4MY9PcMZrCqIoQL?si=94b631da2f6041fa'],
]

playlist_infos = [ [] for _ in range(12) ]
for i in range(len(playlists)):
    for j in range(len(playlists[i])):
        playlist_uri = get_playlist_uri(playlists[i][j])

        playlist_infos[i].append(get_track_ids_and_store_audio_features(access_token, playlist_uri))

with open('data/playlist_data.json', 'w', encoding='utf-8') as f:
    json.dump({ 'data': playlist_infos }, f, ensure_ascii=False, indent=4)


fetching playlist 7BIFvKEvSWfr0c1Ur2PxQ5
fetching playlist 6w2nktImGPS83JiAkG2VzB
fetching playlist 5NZj30WhMY4gnMIGPD8gir
fetching playlist 7LDJsFK7uk53RNxax3z2YC
fetching playlist 0eN9zu6ZezHsNodVtHIlAU
fetching playlist 6unGtxWr0zGP2GJQijrzTW
fetching playlist 3kTgfkfU6GomvrEyNgC7LV
fetching playlist 6SP3BZKRwpp5PX0rI1Rmem
fetching playlist 1AmBxyen2AaSKf6xgl4FAa
fetching playlist 6zut6b4UaLmsxVPvTHAyUC
fetching playlist 3n5J91l343pfMflirtBZhD
fetching playlist 5Wlx45esaqtfimuHPGYaF4
fetching playlist 3GzH2VIpDXQaouhWJeBvYn
fetching playlist 4OUEfwVUeRHzAKW5rGjH3M
fetching playlist 6XmxwdTuFrNQJjhUOEPki3
fetching playlist 3thnEzH8MTVm5t4kyyNVGV
fetching playlist 2k2n89VO9E6uYsxPvatLsk
fetching playlist 4ytPJrzShCo3iftbTgHi4p
fetching playlist 0c9nCMbp3TcR8aLbsyn30N
fetching playlist 5kjhVUgYNyu7zZ1rVEgnAm
fetching playlist 4MJf4EASw6fzzfJlUFv5l0
fetching playlist 3V2DajyxOv0Nnfk0AwjcxV
fetching playlist 2NesB7u53TUuMUTUrkntLL
fetching playlist 0ybME665syNkRpRUIRcKqj
fetching playlis

In [123]:
tracks_table = dynamodb.Table('SPOTIFY_TRACKS')
json_to_write = {'data': [ [] for _ in range(12) ]}

def Decimal(number):
    if float(number)%1==0:
        return int(number)
    return float(number)

with open('data/playlist_data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
    
    for i in range(len(data['data'])):
        zodiac = data['data'][i]


        for j in range(len(zodiac)):
            json_to_write['data'][i].append([])
            
            track_ids = zodiac[j]['track_ids']
            track_ids = list(dict.fromkeys(track_ids))
            map_track_ids = []
            for track_id in track_ids:
                map_track_ids.append({'id': track_id})

            index = 0
            while index < len(track_ids) - 1:
                db_response = dynamodb.batch_get_item(
                    RequestItems={
                        'SPOTIFY_TRACKS': {
                            'Keys': map_track_ids[index:index + 100]
                        }
                    }
                )

                track_received = db_response['Responses']['SPOTIFY_TRACKS']
                for track in track_received:
                    data_to_save = {}
                    properties = ['valence', 'danceability', 'instrumentalness', 'acousticness', 'liveness', 'energy', 'speechiness', 'tempo', 'key', 'loudness']

                    for prop in properties:
                        data_to_save[prop] = float(track[prop])
                    
                    json_to_write['data'][i][j].append(data_to_save)


                index += len(track_received) - 1

    with open('data/raw_playlist_data.json', 'w', encoding='utf-8') as f:
        json.dump(json_to_write, f, ensure_ascii=False, indent=4)